In [4]:
import torch.nn as nn
import matplotlib as mpl
import torch.utils.data as data
import matplotlib.pyplot as plt
#package import
import time
import numpy as np
import torch
import random
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
# from visdom import Visdom
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [8]:
# Hyper parameters
random_seed = 1
learning_rate = 0.0004
batch_size = 4
num_epochs= 2000
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_features = 2500
ratio = 1
num_channel = np.array([4,8,16,32,64,128,256,512])*ratio
test_ratio = 0.85
torch.set_num_threads(8)

# load data (expensive operation)
# path_node_die = 'dieImages01thin.npy'
# path_node_BHF = 'BHFImages01thin.npy'
# path_node_Friction = 'FrictionImages01thin.npy'
# path_node_Clearance = 'ClearanceImages01thin.npy'
# path_node_Thickness = 'ThicknessImages01thin.npy'
path_node_inputs = 'temp/dieImagesZ8.npy'
path_node_target = 'temp/final_target_images/REDONE_SAMPLEStargetImages_Strains_Frame_000.npy'
# samplenum = 603

# Inputs = np.zeros((samplenum,5,256,512))
# Inputs[:,0,:] = np.load(path_node_die)[0:samplenum,:,:]
# Inputs[:,1,:] = np.load(path_node_BHF)[0:samplenum,:,:]
# Inputs[:,2,:] = np.load(path_node_Friction)[0:samplenum,:,:]
# Inputs[:,3,:] = np.load(path_node_Clearance)[0:samplenum,:,:]
# Inputs[:,4,:] = np.load(path_node_Thickness)[0:samplenum,:,:]
# Inputs = np.delete(Inputs, [601,602,603,604,605,606,607,608], 0)
# Inputs = np.load(path_node_inputs)[0:samplenum,:,:]
# Targets = np.load(path_node_target)[0:samplenum,0,:] #Targets: thinning, major strain, minor strain, von mises stress
Inputs = np.load(path_node_inputs)
Targets = np.load(path_node_target) #Targets: thinning, major strain, minor strain, von mises stress

# Data
random.seed(random_seed)
torch.random.manual_seed(random_seed)
np.random.seed(random_seed)
index = [i for i in range(len(Targets))]
random.shuffle(index)

print(Inputs.shape)
print(Targets.shape)
print(index)

(5, 256, 512)
(5, 4, 256, 512)
[2, 3, 4, 0, 1]


In [13]:

Inputs = Inputs[index]
Targets = Targets[index]

Inputs_train=Inputs[:int(len(Targets)*test_ratio),:,:]
Inputs_test=Inputs[int(len(Targets)*test_ratio):,:,:]

Targets_train=Targets[:int(len(Targets)*test_ratio),0,:,:] #only 1st channel kept (thinning image)
Targets_test=Targets[int(len(Targets)*test_ratio):,0,:,:]

class FormDataset(data.Dataset):
    def __init__(self,Inputs,Targets):
        self.Inputs=Inputs
        self.Targets=Targets

    def __getitem__(self,index):
        Input, Target=self.Inputs[index,:,:,:], self.Targets[index,:,:]

        Input=torch.from_numpy(Input).float().to(device)
        Target=torch.from_numpy(Target).float().to(device)

        a = torch.zeros([5,256,512])
        a[:]=Input[:,:,:]
        b = torch.zeros([1,256,512])
        b[0] = Target
        return a, b

    def __len__(self):
        return self.Inputs.shape[0]
    
print('Inputs training dimensions:', Inputs_train.shape)
print('Inputs testing dimensions:', Inputs_test.shape)
print('Targets training dimensions:', Targets_train.shape)
print('Targets testing dimensions:', Targets_test.shape)

#Input and target is same
Formdataset_train=FormDataset(Inputs_train,Targets_train)
Formdataset_test=FormDataset(Inputs_test,Targets_test)

train_loader=torch.utils.data.DataLoader(dataset=Formdataset_train,
                                        batch_size=batch_size,
                                        shuffle=True)

test_loader=torch.utils.data.DataLoader(dataset=Formdataset_test,
                                       batch_size=batch_size,
                                       shuffle=False)

Inputs training dimensions: (4, 256, 512)
Inputs testing dimensions: (1, 256, 512)
Targets training dimensions: (4, 256, 512)
Targets testing dimensions: (1, 256, 512)
